In [1]:
import pickle
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import os
import sys
sys.path.append("../..")
#from DFG_Text_Features.ipynb import *
#from E4525_ML import plots
#plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots

In [2]:
documents=pd.read_csv("filenames.csv",index_col="document_id")
data_dir='preprocessing'

In [3]:
#import spacy
#from html import unescape

# create a spaCy tokenizer
#spacy.load('en')
#lemmatizer = spacy.lang.en.English()

def stem_tokenizer(text):
    porter_stemmer = PorterStemmer()
    return [porter_stemmer.stem(token) for token in word_tokenize(text.lower())]

# remove html entities from docs and
# set everything to lowercase
def my_preprocessor(text):
    if (type(text) is not list):
        text=text.split('\n')
    text=" ".join(text).replace("\n"," ").replace('\u200b', '').replace('\n', '')
    return text

# tokenize the doc and lemmatize its tokens
def my_tokenizer(text):
    porter_stemmer = PorterStemmer()
    punctuation=list(string.punctuation)
    stop0=" ".join(stopwords.words("english")+punctuation)
    stop_words=set(stem_tokenizer(stop0))
    stem_list=stem_tokenizer(text)
    used_list=[token for token in stem_list if token not in stop_words]
    return used_list

# create a dataframe from a word matrix
def wm2df(wm, feat_names):
    
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm.toarray(), index=doc_names,
                      columns=feat_names)
    return(df)




#### Set Features

In [11]:
set_features_filename=data_dir+ "/" + "set_features.p"
set_features=pickle.load(open(set_features_filename,"rb"))

#### Count Features

In [11]:
count_features_filename=data_dir+"/"+"count_features.p"
count_features=pickle.load(open(count_features_filename,"rb"))


In [12]:
#count_test_features_filename=data_dir+"/"+"count_test_features.p"
#count_test_features=pickle.load(open(count_test_features_filename,"rb"))
#count_test_features=normalize(count_test_features)

#### TF-IDF Features

In [13]:
tfidf_features_filename=data_dir+"/"+"tfidf_features.p"
tfidf_vectorizer_filename=data_dir+"/"+"tfidf_vectorizer.p"

tfidf_features=pickle.load(open(tfidf_features_filename,"rb"))
tfidf_vectorizer=pickle.load(open(tfidf_vectorizer_filename,"rb"))

In [10]:
#tfidf_test_features_filename=data_dir+"/"+"tfidf_test_features.p"
#tfidf_test_features=pickle.load(open(tfidf_test_features_filename,"rb"))

In [14]:
tfidf_features

<5x6532 sparse matrix of type '<class 'numpy.float64'>'
	with 12418 stored elements in Compressed Sparse Row format>


## Drop In Model Here

#### SVD Example

In [ ]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(tfidf_features)

terms = tfidf_vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")